# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: XXXX XXXX
#### Student ID: 000000

Date: XXXX

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction
You should give a brief information of this assessment task here.

<span style="color: red"> Note that this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. Note also that any generic comments written in this notebook are to be removed and replace with your own words.</span>

## Importing libraries 

In [34]:
# Code to import libraries as you need in this assessment, e.g.,
import nltk
import os
import string
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
# List of directories containing the text files
directories = [
    r"C:\Users\Kien\Downloads\assess3_data\data\Engineering",
    r"C:\Users\Kien\Downloads\assess3_data\data\Healthcare_Nursing",
    r"C:\Users\Kien\Downloads\assess3_data\data\Accounting_Finance",
    r"C:\Users\Kien\Downloads\assess3_data\data\Sales"
]

# Path to the stop words file
stopwords_file = r"C:\Users\Kien\Downloads\assess3_data\stopwords_en.txt"

# Load stop words
with open(stopwords_file, 'r') as f:
    stopwords = set(f.read().splitlines())

# Function to preprocess text
def preprocess_text(text, stopwords, term_frequencies, top_50_terms):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation (if necessary, but the regex handles this too)
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize using the specific regular expression
    tokens = re.findall(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?", text)
    # Remove words with length less than 2
    tokens = [word for word in tokens if len(word) > 1]
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords]
    # Remove words that appear only once in the document collection
    tokens = [word for word in tokens if term_frequencies[word] > 1]
    # Remove the top 50 most frequent words
    tokens = [word for word in tokens if word not in top_50_terms]
    return ' '.join(tokens)


# Collect term frequencies across all documents
term_frequencies = Counter()
documents = []

# Build the term frequency dictionary
for directory in directories:
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding= 'unicode_escape') as file:
                content = file.read()
                description = content.split("Description: ")[1].strip()
                # Convert description to lowercase and tokenize
                tokens = description.lower().translate(str.maketrans('', '', string.punctuation)).split()
                term_frequencies.update(tokens)
                documents.append(tokens)

# Identify the top 50 most frequent words
top_50_terms = {word for word, _ in term_frequencies.most_common(50)}

# List to store the results
results = []

# Preprocess the text and store the results
for directory in directories:
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding= 'unicode_escape') as file:
                content = file.read()
                title = content.split("Title: ")[1].split("\n")[0]
                webindex = content.split("Webindex: ")[1].split("\n")[0]
                description = content.split("Description: ")[1].strip()
                # Pre-process the Description
                preprocessed_description = preprocess_text(description, stopwords, term_frequencies, top_50_terms)
                # Store the result
                results.append({
                    'Title': title,
                    'Webindex': webindex,
                    'Description': preprocessed_description
                })

# Print the results
for result in results:
    print(result)

{'Title': 'Telesales Operator/ Sales Negotiator', 'Webindex': '71796980', 'Description': 'profitable estate agent recruit telesales communication join prime candidate possess previous negotiating customer estate agency candidate adopt flexible attitude hours evening weekend required full training induction return earn competitive basic salary fantastic bonus structure'}
{'Title': 'Assembly/Production Technicians Milton Keynes', 'Webindex': '71631590', 'Description': 'main purpose perform range mechanical assembly hydraulic installation electrical wiring manufacturing production operations required quality standard cost targets main tasks efficient correct specification mechanical assembly hydraulic installation electrical wiring fabrication fitting operations linked manufacture repair access platforms competent build inspection stages materials full compliance iso procedures total quality focus direct stages manufacturing process achieve quality product aligned customers specification 

## Task 2. Generating Feature Representations for Job Advertisement Descriptions

### Count Vector

In [36]:
# Load the vocabulary from the file
vocab_dict = {}
with open('vocab.txt', 'r') as file:
    for line in file:
        word, idx = line.strip().split(':')
        vocab_dict[word] = int(idx)

In [37]:
# Initialize the CountVectorizer with the custom vocabulary
cvector = CountVectorizer(vocabulary=vocab_dict)

# Extract the preprocessed descriptions and corresponding webindex from the results
descriptions = [result['Description'] for result in results]
webindexes = [result['Webindex'] for result in results]

# Fit and transform the descriptions using the cvector
X_c = cvector.fit_transform(descriptions)

### TF-IDF Vector

In [38]:
# Initialize the TfidfVectorizer with the custom vocabulary
tfidf_vector = TfidfVectorizer(vocabulary=vocab_dict)

# Fit and transform the descriptions using the TfidfVectorizer
X_tfidf = tfidf_vector.fit_transform(descriptions)

### Unweighted Vector

### Saving outputs
Save the count vector representation as per spectification.
- count_vectors.txt

In [39]:
# Open the file to write the sparse count vector representation
output_file_path = 'count_vectors.txt'
with open(output_file_path, 'w') as file:
    # Iterate over each description and its corresponding webindex
    for idx, webindex in enumerate(webindexes):
        # Get the non-zero elements of the sparse matrix row
        sparse_row = X_c[idx]
        non_zero_indices = sparse_row.nonzero()[1]
        sparse_representation = []
        
        for word_idx in non_zero_indices:
            word_count = sparse_row[0, word_idx]
            sparse_representation.append(f"{word_idx}:{word_count}")
        
        # Format the line
        line = f"#{webindex}," + ','.join(sparse_representation) + '\n'
        file.write(line)

print(f"Count vectors exported to {output_file_path}")

Count vectors exported to count_vectors.txt


## Task 3. Job Advertisement Classification

...... Sections and code blocks on buidling classification models based on different document feature represetations. 
Detailed comparsions and evaluations on different models to answer each question as per specification. 

<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [40]:
# Code to perform the task...


## Summary
Give a short summary and anything you would like to talk about the assessment tasks here.

## Couple of notes for all code blocks in this notebook
- please provide proper comment on your code
- Please re-start and run all cells to make sure codes are runable and include your output in the submission.   
<span style="color: red"> This markdown block can be removed once the task is completed. </span>